In [3]:
!pip install bs4

import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

     |████████████████████████████████| 122kB 7.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Libraries imported.


In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

In [6]:
data

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YE1HFmql0OfHi1Vul@x2-wAAARA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1011037969,"wgRevisionId":1011037969,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","Po

In [49]:
neigh_data = pd.DataFrame(columns=["Postal", "Borough", "Neighborhood"])

for row in soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    if len(col) ==3:
        #print(col)
        postal =col[0].text
        pl = len(postal)-1
        postal = postal[:pl]
        borough =col[1].text
        bl = len(borough)-1
        borough = borough[:bl]
        neighborhood =col[2].text
        nl = len(neighborhood)-1
        neighborhood = neighborhood[:nl]
        #print(postal, borough, neighborhood)
        if neighborhood == 'Not assigned':
            neighborhood = borough
        if borough != 'Not assigned':
            neigh_data = neigh_data.append({"Postal":postal, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)
        #neigh_data = neigh_data.append({"Postal":postal, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)
'''
for i in range(0,len(neigh_data)):
    p = neigh_data['Postal'][i]
    for s in range(0,len(neigh_data)):
        if s != i:
            if p == neigh_data['Postal'][s]:
                #print('Combine')
                neigh_data['Neighborhood'][i] = neigh_data['Neighborhood'][i] + ', ' + neigh_data['Neighborhood'][s]
                neigh_data.drop(s)
                s = len(neigh_data)-1
                i = 0
'''
'''
for i in range(0,len(neigh_data)):
    p = neigh_data['Postal'][i]
    for s in range(0,len(neigh_data)):
        if s != i:
            if p == neigh_data['Postal'][s]:
                print('Combine')
'''             
                
                
neigh_data
        

,Postal,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [50]:
neigh_data.shape

(103, 3)

In [52]:
csvf = "Geospatial_Coordinates.csv"

In [54]:
df = pd.read_csv(csvf)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
neigh_data_LL = pd.DataFrame(columns=["Postal", "Borough", "Neighborhood", "Latitude", "Longitude"])

for i in range(0, len(neigh_data)):
    postal = neigh_data['Postal'][i]
    borough = neigh_data['Borough'][i]
    neighborhood = neigh_data['Neighborhood'][i]
    for r in range(0, len(df)):
        if df.iloc[i,0] == postal:
            latitude = df.iloc[i, 1]
            #print(df[i][1])
            longitude = df.iloc[i, 2]
    neigh_data_LL = neigh_data_LL.append({"Postal":postal, "Borough":borough, "Neighborhood":neighborhood, "Latitude": latitude, "Longitude": longitude}, ignore_index=True)
    
    
neigh_data_LL

,Postal,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.605647,-79.501321
1,M4A,North York,Victoria Village,43.605647,-79.501321
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.605647,-79.501321
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.605647,-79.501321
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.605647,-79.501321
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.605647,-79.501321
99,M4Y,Downtown Toronto,Church and Wellesley,43.605647,-79.501321
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.605647,-79.501321
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.605647,-79.501321


In [89]:
b_list = []

for i in range(0, len(neigh_data_LL)):
    if neigh_data_LL['Borough'][i].find('Toronto') != -1:
        #print(neigh_data_LL['Borough'][i])
        if neigh_data_LL['Borough'][i] in b_list:
            a=0
        else:
            b_list.append(neigh_data_LL['Borough'][i])
print(b_list)

toronto_data = pd.DataFrame(columns=["Postal", "Borough", "Neighborhood", "Latitude", "Longitude"])

for i in range(0, len(b_list)):
    td_i = neigh_data_LL[neigh_data_LL['Borough'] == b_list[i]].reset_index(drop=True)
    toronto_data = toronto_data.append(td_i, ignore_index=True)
toronto_data.head()



['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto', 'Toronto/York']


,Postal,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.605647,-79.501321
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.605647,-79.501321
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.605647,-79.501321
3,M5C,Downtown Toronto,St. James Town,43.605647,-79.501321
4,M5E,Downtown Toronto,Berczy Park,43.605647,-79.501321


In [91]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [94]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto